***

#### System transfer function (s-domain)

Let's start with a second-order low-pass system defined by its transfer function with

$$ H(s) = \frac{G \omega _{n}^2}{s^2+2\zeta\omega_{n}s+\omega_{n}^2} $$ 

where $G$ is the system static gain, $\omega_{n}$ is the natural frequency, and $\zeta$ is the damping parameter.

If we include a pure delay, $\delta $, we get

$$ H(s) = \frac{G \omega _{n}^2}{s^2+2\zeta\omega_{n}s+\omega_{n}^2}e^{-\delta s} $$ 

#### System impulse and step response functions (time domain)

The inverse Laplace transform of the system transfer function is the parametric system impulse response function (we could use the python symbolic package, sympy, to do this). The function defined below is the general form of a parametric second-order, low-pass impulse response function:

$$ h(\tau) = \frac{e^{-\omega_{n}\tau\zeta}G\omega_{n}sinh(\omega_{n}\tau\sqrt{\zeta^2-1})}{\sqrt{\zeta^2-1}} $$

And if we include a pure delay, $\delta$, we get

$$ h(\tau) = \frac{e^{-\omega_{n}(\tau-\delta)\zeta}G\omega_{n}sinh(\omega_{n}(\tau-\delta)\sqrt{\zeta^2-1})}{\sqrt{\zeta^2-1}} $$

We now define a python functions, to evaluate $H(s)$ and $h(\tau)$ 

In [ ]:
import csv

with open('filename.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        print(row)

In [1]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # this is key to improving plot resolution

In [2]:
def tf(s,gain,damping,natfreq,delta):
    return gain*natfreq**2/(s**2+2*damping*natfreq*s+natfreq**2)*np.exp(-delta*s)

In [3]:
def irf2(tau, gain, damping, natfreq, delta):
    # evaluates a second order low pass impulse reponse function at lags, tau
    # note that natfreq is in rad/s
    if damping==1: # special case when damping == 1 (to avoid division by 0)
        damp=1.00000001
    else:
        damp=damping
    d=np.sqrt(complex(-1.0 + damp**2)) # note that complex is required to force np.sqrt to return a complex number
    ir2=np.real((np.exp(-(tau-delta) * damp * natfreq) * gain * natfreq * np.sinh((tau-delta) * natfreq * d))/d)
    return np.heaviside(tau-delta,0.5)*ir2

In [4]:
def srf2(tau, gain, damping, natfreq, delta):
    # evaluates a second order low pass unit step response function at lags, tau
    # note that natfreq is in rad/s
    if damping==1: 
        d=np.exp(-(tau-delta)*natfreq)
        sr2=gain*(1-d*(1+natfreq*(tau-delta)*d))
    elif damping<1:
        d=np.sqrt((complex(1-damping**2))) # note that complex is required to force np.sqrt to return a complex number
        sr2=gain*(1-(np.exp(-damping*(tau-delta)*natfreq)/d)*np.sin(natfreq*d*(tau-delta)+np.arcsin(d)))
    else: # damping >1
        d=np.sqrt((complex(damping**2-1))) # note that complex is required to force np.sqrt to return a complex number
        sr2=gain*(1+(1/(2*(damping+d)*d))*np.exp(-natfreq*(damping+d)*(tau-delta))-(1/(2*(damping-d)*d))*np.exp(-natfreq*(damping-d)*(tau-delta)))
    return np.heaviside(tau-delta,0.5)*np.real(sr2)

In [5]:
def plot_irf2(gain,damping,natfreq,delta):
    
    tau=np.arange(0.0, 20, 0.001)
    h=irf2(tau,gain,damping,natfreq,delta)
    sr=srf2(tau,gain,damping,natfreq,delta)
    
    w1=natfreq*0.01
    w2=natfreq
    w3=natfreq*100
    g1=np.absolute(tf(1j*w1,gain,damping,natfreq,delta))
    g2=np.absolute(tf(1j*w2,gain,damping,natfreq,delta))
    g3=np.absolute(tf(1j*w3,gain,damping,natfreq,delta))   
    w=np.logspace(0, 4, 1000)
    tfg=np.absolute(tf(1j*w,gain,damping,natfreq,delta)) # substitute s for jw
    tfp=360*np.unwrap(np.angle(tf(1j*w,gain,damping,natfreq,delta)))/(2*np.pi) # substitute s for jw
    
    fig, (ax1a, ax2a) = plt.subplots(1, 2)
    fig.suptitle('Second-order Low-pass System (natural freq in rad/s)')
    #ax1.plot(x, y)
    #ax2.plot(x, -y)
    
    plt.rcParams['figure.figsize'] = [9, 6]
    
    #fig, ax1 = plt.subplots()
    ax1a.plot(tau, h, 'r',linewidth=1)
    ax1a.set_ylim([-200,200])
    ax1a.set_xlim([0,1])
    ax1a.set(xlabel='Time (s)', ylabel='Impulse Response Amplitude (red)', title='Time Domain')
    ax1a.grid()
    
    ax1b=ax1a.twinx()
    ax1b.plot(tau,sr,'b',linewidth=1)
    ax1b.set(ylabel='Step Response Amplitude (blue)')
    ax1b.set_ylim([-4,4])
    
    ax2a.loglog(w/(2*np.pi), tfg,'r',linewidth=1)
    ax2a.loglog([w1/(2*np.pi),w2/(2*np.pi)],[gain,gain],'b:',linewidth=1) 
    ax2a.loglog([w2/(2*np.pi),w3/(2*np.pi)],[gain,g3],'b:',linewidth=1)
    ax2a.set_ylim([0.001,10])
    ax2a.set_xlim([0.1,1000])
    ax2a.set(xlabel='Frequency (Hz)', ylabel='Gain (red)', title='Frequency Domain')
    #ax2a.set_yticks(np.arange(-4,2,1))
    #ax2a.grid()
    
    ax2b=ax2a.twinx()
    ax2b.semilogx(w/(2*np.pi), tfp,'b',linewidth=1)
    ax2b.semilogx([0.1,1000],[-180,-180],'k:',linewidth=1)
    ax2b.semilogx([0.1,1000],[-90,-90],'k:',linewidth=1)
    ax2b.semilogx([0.1,1000],[0,0],'k:',linewidth=1)
    ax2b.semilogx([0.1,1000],[90,90],'k:',linewidth=1)
    ax2b.semilogx([w2/(2*np.pi),w2/(2*np.pi)],[-270,180],'k:',linewidth=1)
    ax2b.set(ylabel='Phase (blue)')
    ax2b.set_ylim([-270,180])
    ax2b.set_xlim([0.1,1000])
    ax2b.locator_params(axis = 'y', nbins = 8)
    ax2b.set_yticks(np.arange(180,-270,-90))
    
    plt.tight_layout()
    
    plt.show()

In [6]:
interact(plot_irf2, 
         gain = widgets.FloatSlider(value=2, min=-4.0, max=4.0,step=0.1,layout=Layout(width='700px')), 
         damping = widgets.FloatSlider(value=0.3, min=0.02, max=2.0,step=0.01,layout=Layout(width='700px')),
         natfreq = widgets.FloatSlider(value=10*2*np.pi, min=10, max=200,step=1,layout=Layout(width='700px')),
         delta = widgets.FloatSlider(value=0.0, min=0.0, max=0.02,step=0.0001,layout=Layout(width='700px'),readout_format='.4f'))

interactive(children=(FloatSlider(value=2.0, description='gain', layout=Layout(width='700px'), max=4.0, min=-4…

<function __main__.plot_irf2(gain, damping, natfreq, delta)>